# Overview

At the end of the last chapter, we concluded that DP was model-based. This means that it didn't need to actually play any episodes - we were in "God-mode". However, for many scenarios such as self-driving cars and video-games, it's unreasonable to think that you'd know everything about the environment. Monte Carlo methods on the other hand learn puely from experience.

Monte Carlo usually refers to any method with a significant random component. In RL, the random component is the return. With MC we don't calculate the true expected value of the return ($G$) - instead we calculate its sample mean.

Before we can calculate returns, the episode must terminate, hence MC only works with episodic tasks. Also due to this, MC isn't "fully online" ie. improvements happen after episodes rather than in real time (after every action).

This is similar to the multi-armed bandits of Chapter 1. With that, we'd always average the reward after every episode. With MDPs we're always averaging the return.

We'll follow the same pattern as last chapter, first investigating the prediction problem, then moving on to the control problem.

# MC Policy Evaluation

Recall that

$$V_\pi(s) = E \left[ G(t) \mid S_t=s \right]  .$$

Also note that for $i=$ episode and $s=$ state,

$$\overline{V}_\pi(s)=\dfrac{1}{N}\sum_{i=1}^NG_{i,s}  .$$

In order to calculate $G$, we just play a number of episodes, record the states and reward sequences. Then we can calculate $G$ from the definition (iterating through the states in reverse order since $G$ depends only on future values),

$$G(t) = r(t+1) + \gamma\left( G(t+1) \right)  .$$

Once we have the pairs $(s,G)$, we just average them for each $s$ (ie. take the sample mean).

## Multiple visits to $s$

If we see a state $s$ more than once in an episode, ie. we see $s$ at $t=1$ and $t=3$, we can either only include the first instance in our sample mean (first-visit MC), or include them all (every-visit MC). It has been proven that they both lead to the same answer.

## Pseudocode

In [ ]:
def first_visit_mc_prediction(policy, N):
    
    V = random_initialisation
    all_returns = {}
    
    do N times:
        states, returns = play_episode
        
        for s, g in zip(states,returns):
            if not seen s this episode yet:
                all_returns[s].append(g)
                V(s) = sample_mean(all_returns[s])
                
    return V

## Sample Mean Calculations

Recall that in Chapter 1 we discussed more efficient ways of calculating the sample mean. Everything learnt there can be applied here.

The rules of probability still apply:
<ul>
    <li>The confidence interval is approximately Gaussian (Central Limit Theorem).</li>
    <li>The variance is the original variance of the data divided by the number of samples collected. Hence we are going to be more confident the more samples we have, but growth gets slower the more we have.</li>
</ul>

## Calculating Returns from Rewards - Pseudocode

In [ ]:
state = env.return_pos()

states_and_rewards = [(state, 0)]
while not env.game_over:
    action = policy(state)
    reward = env.move(action)
    state = env.return_pos()
    states_and_rewards.append((s, r))

G = 0
states_and_returns = []
for state, reward in reverse(states_and_rewards):
    states_and_returns.append((state, G))
    G = reward + GAMMA * G

states_and_returns.reverse()

## An Advantage of MC

Recall that one disadvantage of DP is that we need to loop through all states. For MC, we only update $V$ for visited states - we don't need to know what all the states are, we can just discover them as we play.